# Scraping et formatage des données DATAN

<div style="text-align: right"> André Mounier </div>

---
Application sur les données DATAN

In [1]:
import os
import pandas as pd
import tqdm
from pathlib import Path
import numpy as np
from collections import Counter

### Liste des votes

In [3]:
df_allvotes = pd.read_html('https://datan.fr/votes/legislature-16')[0]
df_allvotes = df_allvotes.set_index('N°')

In [4]:
df_allvotes.iloc[3702]

Date                                               27-10-2022
Titre       L'amendement n° 1882 de M. Schreck à l'article...
Résultat                                               REJETÉ
Name: 403, dtype: object

### Tests de récupération des détails des votants (groupes et parlementaires)

In [5]:
vote_number = 403

In [6]:
vote_infos = pd.read_html('https://datan.fr/votes/legislature-16/vote_{}'.format(vote_number))
df_desc, _, df_groups, df_deputees = vote_infos[:4]
# df_deputees

### Création d'un unique tableau de données

In [7]:
output_path = os.path.join('output','DATAN')
Path(output_path).mkdir(parents=True, exist_ok=True)

In [8]:
dict_group = {'ni':'Non inscrit (NI)',
              'dem':'Démocrate (MoDem et Indépendants) (DEM)',
              'lfi-nupes':'La France insoumise - NUPES (LFI-NUPES)',
              'soc-a':'Socialistes et apparentés - NUPES (SOC)',
              'gdr-nupes':'Gauche démocrate et républicaine - NUPES (GDR-NUPES)',
              'lr':'Les Républicains (LR)',
              'hor':'Horizons et apparentés (HOR)',
              'rn':'Rassemblement National (RN)',
              'ecolo':'Écologiste - NUPES (ECOLO)',
              'liot':'Libertés, Indépendants, Outre-mer et Territoires (LIOT)',
              're':'Renaissance (RE)'}

In [11]:
if 'datan.csv' not in os.listdir(output_path):

    # initilisation des colonnes du tableau de sortie
    columns = {'vote':[],'date':[],'type':[],'dossier':[]}
    for group in dict_group.keys():
        columns['{}_nb-pour'.format(group)] = []
        columns['{}_nb-contre'.format(group)] = []
        columns['{}_nb-abstention'.format(group)] = []
        columns['{}_participation'.format(group)] = []
    
    # construction du tableau
    for vote in tqdm.tqdm(df_allvotes.index):
        vote_infos = pd.read_html('https://datan.fr/votes/legislature-16/vote_{}'.format(vote))
        df_desc, _, df_groups, df_deputees = vote_infos[:4]
        
        # formatage du tableau des votes de groupe
        df_groups_formatted = df_groups.set_index('Groupe')
        
        # formatage du tableau descriptif du vote
        df_desc_formatted = df_desc.T
        df_desc_formatted = df_desc_formatted.rename(columns={i:n for i,n in enumerate(df_desc_formatted.iloc[1])})
        df_desc_formatted
        
        # ajout de la date
        try:
            date = df_desc_formatted['Date'].values[-1]
        except KeyError:
            date = np.nan
        
        # ajout du type de vote 
        try:
            type_vote = df_desc_formatted['Type de vote'].values[-1]
        except KeyError:
            type_vote = np.nan
            
        # ajout du dossier
        try:
            dossier = df_desc_formatted['Dossier'].values[-1]
        except KeyError:
            dossier = np.nan
            
        columns['vote'].append(vote)
        columns['date'].append(date)
        columns['type'].append(type_vote)
        columns['dossier'].append(dossier)
        
        # ajout des statistiques de votes par groupe parlementaire
        for group, group_name in dict_group.items():
            filter_group = df_groups_formatted.index.str.startswith(group_name[:10])
            df_group = df_groups_formatted.loc[filter_group]
            nb_pour, nb_contre, nb_abstention, ratio_particip = df_group[['Pour','Contre','Abstention','Participation']].values[0]
            ratio_particip = float(ratio_particip.replace('%',''))/100
            
            columns['{}_nb-pour'.format(group)].append(nb_pour)
            columns['{}_nb-contre'.format(group)].append(nb_contre)
            columns['{}_nb-abstention'.format(group)].append(nb_abstention)
            columns['{}_participation'.format(group)].append(ratio_particip)

    # sauvegarde
    datan = pd.DataFrame().from_dict(columns)
    datan.to_csv(os.path.join(output_path,'datan.csv'),index=False)

### Ouverture et manipulation du jeu de données

In [13]:
datan = pd.read_csv(os.path.join(output_path,'datan.csv'))

In [15]:
counter_dict = dict(Counter(datan.dossier))

# exclusion des sans dossiers et des projets de lois de finances
counter_dict = {k:v for k,v in counter_dict.items() if not any([e in str(k).lower() for e in ['finances','nan']])}

print(len(counter_dict))

counter_dict

138


{'Accompagnement des malades et fin de vie': 123,
 "Proposition de loi visant à poursuivre la dématérialisation de l'état civil du ministère de l'Europe et des affaires étrangères": 1,
 'Prévenir les ingérences étrangères en France': 23,
 "Prise en charge intégrale des soins liés au traitement du cancer du sein par l'assurance maladie": 4,
 "Meilleure réussite scolaire des jeunes ultramarins grâce à l'apprentissage des langues régionales": 2,
 'Constitutionnaliser la sécurité sociale': 2,
 'Souveraineté en matière agricole et renouvellement des générations en agriculture': 108,
 'Assurer une justice patrimoniale au sein de la famille': 2,
 'Projet de loi constitutionnelle portant modification du corps électoral pour les élections au congrès et aux assemblées de province de la Nouvelle-Calédonie': 9,
 "Améliorant l'efficacité des dispositifs de saisie et de confiscation des avoirs criminels": 7,
 'Proposition de loi visant à améliorer le repérage et l’accompagnement des personnes présen